# December 2017: Advent of Code Solutions
## Daniel Näslund
From Dec. 1 to Dec. 25, [I](dannas.name) will be solving the puzzles that appear each day at [Advent of Code](http://adventofcode.com/). The two-part puzzles are released at midnight EST (6:00AM CET); points are awarded to the first 100 people to solve the day's puzzles. 

To understand the problems completely, you will have to read the full description in the "[Day 1](http://adventofcode.com/2017/day/1):" link in each day's section header.

##  Prelude
Here I import common functions and modules so I don't have to do it each day.
These are borrowed from [Peter Norvigs Advent of Code solutions](https://github.com/norvig/pytudes/blob/master/ipynb/Advent%20of%20Code.ipynb) from 2016. I've also reused his ipython notebook layout.

In [1]:
# Python 3.x
from itertools import islice, cycle, permutations
import operator

cat = ''.join
def Input(day):
    filename = 'advent2017/input{}.txt'.format(day);
    return open(filename);
    # TODO(dannas): Fetch the files from elsewhere to allow remote access
    
# 2-D points implemented using (x, y) tuples
def X(point): return point[0]
def Y(point): return point[1]

def cityblock_distance(p, q=(0, 0)): 
    "City block distance between two points."
    return abs(X(p) - X(q)) + abs(Y(p) - Y(q))

# [Day 1](http://adventofcode.com/2017/day/1): Inverse Captcha
Given a file of digits, find the sum of all digits that match the next digit in the list. The list is circular, so the digit after the last digit is the first digit in the list.

We need to parse with one token lookahead, it's enough to append the first digit to the end of the list for handling the circular case.

In [ ]:
def pairs(seq):
    return zip(seq[:-1], seq[1:])

def solve_captcha(str):
    digits = [int(d) for d in str] 
    if len(digits) < 2:
        return 0
    digits.append(digits[0])
    return sum(x for x, y in pairs(digits) if x == y)
    
solve_captcha(Input(1).read().strip())

In **part 2** we shall compare the digit halfway around the circular list to the current one. The list has an even number of elements.

This require N/2 token lookahead instad of one. I could have appended the first half of the list to the end, but instead I opted for a circular queue. A circular queue can be implicitely represented using indexes that wrap around; or explicitely using an [ADT](https://docs.python.org/3.6/library/collections.html?highlight=deque#collections.deque.rotate); or using operations on iterators. I choose the later.

The [cycle()](https://docs.python.org/3.6/library/itertools.html#itertools.cycle) function provides an iterator to an infinite circular representation of the list. With [islice](https://docs.python.org/3.6/library/itertools.html#itertools.islice) I can select my start and end position in that list.

In [ ]:
def pairs(seq):
    N = len(seq)
    half = int(N/2)
    x = islice(cycle(seq), 0, N)
    y = islice(cycle(seq), half, N + half)
    return zip(x, y)

def solve_captcha(str):
    digits = [int(d) for d in str] 
    return sum(x for x, y in pairs(digits) if x == y)
    
solve_captcha(Input(1).read().strip())


## [Day 2](http://adventofcode.com/2017/day/2): Corruption Checksum
For each row in a spreadsheet, determine the difference between largest and smallest value; the checksum is the sum of all of these differences.

I initially had some trouble parsing the input into a list of list of integers. It's been a while since I used Python.

In [ ]:
def maxmins(spreadsheet):
    for row in spreadsheet:
        yield max(row), min(row)

def parse(line):
    return tuple(int(x) for x in line.split())

spreadsheet = [parse(line) for line in Input(2)]
sum(x-y for x,y in maxmins(spreadsheet))

In **part 2** we're asked to find the only two numbers in each row where one evenly divides the other - that is, where the result of the division operation is a whole number. Find those numbers on each line, divide them, and add up each line's result.

In [ ]:
def evens(spreadsheet):
   for row in spreadsheet:
        for x, y in permutations(row, 2):
            if x % y == 0:
                yield x, y
                
def parse(line):
    return tuple(int(x) for x in line.split())

spreadsheet = [parse(line) for line in Input(2)]
sum(x/y for x,y in evens(spreadsheet))


## [Day 3](http://adventofcode.com/2017/day/3): Spiral Memory
TODO(dannas): Add description

In [11]:
# How represent position? tuple (x,y) or complex number?
# How move? A while loop and states?
# Each side increments with step=2
# E,N starts on 1
# W,S starts on 2
#
# When in doubt, use brute force - Ken Thompsson
# A while loop that steps down one step
# state

E, N, W, S = [(1, 0), (0, 1), (-1, 0), (0, -1)]    

def move(pos, direction):
    return (X(pos) + X(direction), Y(pos) + Y(direction))

def turn(direction, num_steps):
    changes = {
        E : (N, 0),
        N : (W, 1),
        W : (S, 0),
        S : (E, 1),
    }
    d, delta = changes[direction]
    return d, num_steps + delta


def walk(total_steps):
    pos = (0, 0)
    num_steps = 1
    walked = 0
    direction = E
    
    while total_steps > 0:
        walked += 1
        pos = move(pos, direction)
        if walked == num_steps:
            direction, num_steps = turn(direction, num_steps)
            walked = 0

        total_steps -= 1
    return pos

pos = walk(289326 - 1)

cityblock_distance(pos)

419

**part 2** TODO(dannas): Solve this task!!!

In [14]:
# We have positions and values
# Position can be represented using a tuple
# value can be a mapping from tuples to val  (x, y) => val
# We want to know the neighbours values
# So we need  neighbours() function
# Looks like corners has 2 neighbours
#            the other ones have 3 neighbours
# Special case: The first two positions has only one value

# For determining neighbours I need to know the current direction
# How about representing directions as unit vectors?
# That vector can be represented using a tuple

square_values = {}

# TODO(dannas): Figure out the pattern for which unit vectors that
# represents neighbours
def neighbour_sum(pos, direction):
    nsum = 0
    mappings = {
        W : [(0, -1), (-1, -1), ( 1, 0)],
        E : [(0,  1), (-1,  1), (-1, 0)],
        N : [(0, -1), (-1, -1), (-1, 0)],
        S : [(0,  1), ( 1,  0), ( 1, 1)]
    }
    
    for n in mappings[direction]:
        np = move(pos, n)
        if np in square_values:
            nsum += square_values[np]
    return nsum
   
def walk(lower_limit):
    pos = (0, 0)
    square_values[pos] = 1
    num_steps = 1
    walked = 0
    direction = E
    
    while square_values[pos] <= lower_limit:
        walked += 1
        pos = move(pos, direction)
        square_values[pos] = neighbour_sum(pos, direction)
        if walked == num_steps:
            direction, num_steps = turn(direction, num_steps)
            walked = 0
    return square_values[pos]

walk(289326)

307856